In [ ]:
print('Gajraj')

In [ ]:
# !pip install openai langchain_google_genai duckdb
# !pip show openai
# !pip install kor

In [30]:
import os
from langchain.llms.google_palm import GooglePalm
import requests
import duckdb
import pandas as pd
import time

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = "mt

llm = GooglePalm(model="gemini-pro")

def get_ip_address(llm,detail):
    template=f'''
    Can you fetch only ip address related data from following details and put this detail in response field only,
    I need only ip address,i don't need any other details,so don't need to be verbose only gives response in format of ip address?

    ###Details###
        {detail}

    ###Response####

    '''
    result = llm(template)
    return result

def is_gclid_exist(llm,detail):
    template=f'''
    Can you check weather gclid exists from following details or not if you found such detail put this detail in response field only like yes,
    I need only yes and no response,i don't need any other details,so don't need to be verbose only gives response in format of yes or no?

    ###Details###
    {detail}

    ###Response####

    '''

    result = llm(template)
    return result

def get_country(ip_address):
    try:
        response = requests.get(f'http://ip-api.com/json/{ip_address}')
        js = response.json()
        country = js['country']
        return country
    except Exception as e:
        return str(e)

def get_url(llm,detail):

    # def get_ip_address(llm,detail):
    template=f'''
    Can you fetch url path start with /landing from following details and put this detail in response field only,
    I need only url path,i don't need any other details,so don't need to be verbose only gives response in format of url path?
    Response should be like following way

    /landing/flutter-2
    /landing/images
    ###Details###
        {detail}

    ###Response####

    '''
    result = llm(template)
    return result

In [31]:

import pandas as pd
import time
# Create a sample DataFrame
df = pd.DataFrame({
    'ip_address': [],
    'is_gclid': [],
    'url': [],
    'country_name': [],
})

df_data=pd.read_csv('./accesslogs19-01-2023-10-00.csv')

for index,row in df_data.iterrows():
    try:
        if 'gclid' in row[0]:
            time.sleep(5)
            ip_address = get_ip_address(llm,row[0])
            is_gclid = 'yes'
            country_name= get_country(ip_address)
            url= get_url(llm,row[0])
            if ":" not in ip_address:
                print(index)
                new_record = pd.DataFrame([{
                'ip_address': ip_address,
                'url': url.replace('```','').replace('###Response####','').replace('"','').strip(),
                'is_gclid': is_gclid,
                'country_name': country_name
                }])
            df = pd.concat([df, new_record], ignore_index=True)  
            # Insert into db  
    except Exception as ex:
        print(f'Error -> {ex}')
        continue        
    

17
20
21
29
30
31
32
Error -> list index out of range
Error -> list index out of range
Error -> list index out of range
Error -> list index out of range
37
Error -> list index out of range
41
Error -> list index out of range
43
44
45
46
47
48
Error -> list index out of range
50
51
Error -> list index out of range
670
706
710
711
712
714
715
716
770
825
834
867
879
903
930
993
1030
1057
1106


In [32]:
import duckdb
import pandas as pd

con = duckdb.connect(database="./detail.duckdb",read_only=False)

# Create a table named "my_table" from the DataFrame
con.execute("CREATE TABLE click_details AS SELECT * FROM df")

# Insert into the table "my_table" from the DataFrame
con.execute("INSERT INTO click_details SELECT * FROM df")

# Close the connection
con.close()

In [33]:
df.to_csv("./details.csv",index=False)

In [34]:
df.columns

Index(['ip_address', 'is_gclid', 'url', 'country_name'], dtype='object')

In [28]:
import duckdb
dd=duckdb.query("select count(ip_address) as no_of_click,ip_address,is_gclid,country_name,url from df group by ip_address,is_gclid,country_name,url").to_df()

In [29]:
dd.to_csv("details_with_click.csv",index=False)